### MarineTraffic AIS - Vessel Positions API


API documentation https://servicedocs.marinetraffic.com/tag/MarineTraffic-AIS#operation/exportvessels______

API key can be found at https://www.marinetraffic.com/en/users/my_account/api/account



In [ ]:
import io
import csv
import json
import time
import requests
import tempfile, shutil
from datetime import datetime

In [ ]:
# API business configuration

call_limit = 120 #Call frequency allowance in senconds, adjust accordingly
first_request_timestamp = None

In [ ]:
#API parameters

limit = 1000 #limit of vessels per page
timespan = 5 # number of minutes API looks back for available positions
api_key = 'API_KEY' #your full API key

In [ ]:
# API call schema
url = f'https://services.marinetraffic.com/api/exportvessels/{api_key}/v:9/timespan:{timespan}/limit:{limit}/protocol:jsono'

In [ ]:
"""
    getAISPositions() makes one or more calls within API at given URL
"""

## Write data to csv
def write_data_to_csv(f, data):
    if not data:
        return
        
    cols = list(data[0].keys()) # column names
    file_writer = csv.DictWriter(f, dialect='custom', fieldnames=cols)

    if f.tell() == 0: # is file empty
        file_writer.writeheader() # write columns headers once
    file_writer.writerows(data)

def getAISPositions(url,cursor = None):
    request_time = datetime.now()
    data = []
    
    url_with_cursor = f'{url}/cursor:{cursor}' if cursor else url
    
    try:        
        response = requests.get(url_with_cursor)

        if response.ok:
            data = response.json()
        
        response.raise_for_status()
            
    except requests.exceptions.HTTPError as e:
        # Print detailed error message if HTTP error occurs
        print(f"\nHTTP error occurred: {e}")
        print(f"Response content: {response.content.decode('utf-8')}")
    except requests.exceptions.RequestException as e:
        # Handle other request exceptions
        print(f"\nAn error occurred: {e}")
        
    return (request_time,data,url_with_cursor)

In [ ]:
### frquency call restriction
is_allowed = True

#business rule
if first_request_timestamp is not None: #not first request
    request_diff = (datetime.now()-first_request_timestamp).total_seconds()
    
    if(request_diff < call_limit):
        is_allowed = False
    
i = 0

url = f'https://services.marinetraffic.com/api/exportvessels/{api_key}/v:9/timespan:{timespan}/limit:{limit}/protocol:jsono'

##csv setup output
csv.register_dialect('custom', delimiter=';',doublequote=True,quotechar='"',lineterminator='\n',escapechar='\\',quoting=csv.QUOTE_MINIMAL)

# Handle cursor paginated request the request
has_next_cursor = True

if is_allowed:
    try:
        ## file definition
        c_time = int(time.time())
        api_key_last = api_key[-8:] #last 8 characters of API key
        file_name_output = f'out-api-{api_key_last}-{c_time}.csv'
        
        print(f"CSV file:{file_name_output}")
    
        # temp files to avoid losting data in case of memory issues
        f = tempfile.NamedTemporaryFile(suffix='.csv',prefix=f'api-{api_key_last}-',mode='w+', delete=False)
    
        ##handle the request
        cursor = None

        while (is_allowed) and (has_next_cursor):
            req_timestamp,content,url_adjusted = getAISPositions(url,cursor)
        
            if i == 0: #first request
                first_request_timestamp = req_timestamp
                
            data = content.get('DATA', [])
            headers = content.get('METADATA', [])

            print(f'\nRequest {url_adjusted} \nLines: {len(data)}, at {req_timestamp.strftime('%Y-%m-%d %H:%M:%S')}\n')

            write_data_to_csv(f, data)
            
            if('CURSOR' in headers and headers['CURSOR']):
                has_next_cursor = True
                cursor = headers['CURSOR']
                time.sleep(1)
            else:
                has_next_cursor = False
    
            i+=1
            
    except Exception as e:
        print(f"Error during _request: {e}")
        is_allowed = False
    finally:
        if (f is not None) and (f.__class__.__name__ == '_TemporaryFileWrapper'):
            tmp_file_name = f.name
            f.close()
            shutil.move(tmp_file_name, f'./{file_name_output}') #move temp named file to current dir
            
            print(f"\n\nSuccess:CSV at {file_name_output}, moved from temp path {tmp_file_name}")
        else:
            print(f"\n\nTemporary CSV saved at {tmp_file_name}")
else:
    eltime = round((datetime.now()-first_request_timestamp).total_seconds(),2)
    print(f'Next call allowed after {call_limit} seconds. Current: {eltime} seconds.')
